In [1]:
%%capture
pip install pandas

In [2]:
%%capture
pip install great_expectations

In [3]:
import pandas as pd
import great_expectations as ge

In [4]:
df_test = pd.read_csv ('./sales-record.csv')
print(df_test.head())

                         Region  Country        Item Type Sales Channel  \
0         Australia and Oceania    Palau  Office Supplies        Online   
1                        Europe   Poland        Beverages        Online   
2                 North America   Canada           Cereal        Online   
3                        Europe  Belarus           Snacks        Online   
4  Middle East and North Africa     Oman           Cereal       Offline   

  Order Priority Order Date   Order ID  Ship Date  Units Sold  Unit Price  \
0              H   3/6/2016  517073523  3/26/2016        2401      651.21   
1              L  4/18/2010  380507028  5/26/2010        9340       47.45   
2              M   1/8/2015  504055583  1/31/2015         103      205.70   
3              C  1/19/2014  954955518  2/27/2014        1414      152.58   
4              H  4/26/2019  970755660   6/2/2019        7027      205.70   

   Unit Cost  Total Revenue  Total Cost  Total Profit  
0     524.96     1563555.21  1

In [5]:
df_test.columns

Index(['Region', 'Country', 'Item Type', 'Sales Channel', 'Order Priority',
       'Order Date', 'Order ID', 'Ship Date', 'Units Sold', 'Unit Price',
       'Unit Cost', 'Total Revenue', 'Total Cost', 'Total Profit'],
      dtype='object')

In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 14 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Region          object 
 1   Country         object 
 2   Item Type       object 
 3   Sales Channel   object 
 4   Order Priority  object 
 5   Order Date      object 
 6   Order ID        int64  
 7   Ship Date       object 
 8   Units Sold      int64  
 9   Unit Price      float64
 10  Unit Cost       float64
 11  Total Revenue   float64
 12  Total Cost      float64
 13  Total Profit    float64
dtypes: float64(5), int64(2), object(7)
memory usage: 534.1+ MB


In [7]:
df_test['Order ID'].astype(str).astype(int)

0          517073523
1          380507028
2          504055583
3          954955518
4          970755660
             ...    
4999995    940436398
4999996    407689177
4999997    727000367
4999998    714043796
4999999    604805791
Name: Order ID, Length: 5000000, dtype: int64

In [8]:
df_chunk = pd.read_csv ('./sales-record.csv', chunksize = 1000000)
print (df_chunk)

In [9]:
i= 0 
chunk_list = [] 

In [10]:
def chunk_preprocessing(chunk,column):
    return chunk[[column,'Country','Unit Price','Order ID','Order Date']]

In [11]:
def create_df(df_chunk,chunk_list,i):
    for chunk in df_chunk:
        chunk_filter = chunk_preprocessing(chunk,'Region')
        i+=100000
        print(i)
        chunk_list.append(chunk_filter)
    return chunk_list

In [12]:

chunk_list = create_df(df_chunk,chunk_list,i)

data_df = pd.concat(chunk_list)
data_df['Order ID'].astype(str).astype(int)
data_df.index = range(len(data_df))


100000
200000
300000
400000
500000


In [13]:
data_df

,Region,Country,Unit Price,Order ID,Order Date
0,Australia and Oceania,Palau,651.21,517073523,3/6/2016
1,Europe,Poland,47.45,380507028,4/18/2010
2,North America,Canada,205.70,504055583,1/8/2015
3,Europe,Belarus,152.58,954955518,1/19/2014
4,Middle East and North Africa,Oman,205.70,970755660,4/26/2019
...,...,...,...,...,...
4999995,Middle East and North Africa,Iraq,668.27,940436398,3/17/2014
4999996,Europe,Monaco,421.89,407689177,11/8/2015
4999997,Australia and Oceania,Solomon Islands,109.28,727000367,6/1/2020
4999998,Australia and Oceania,Marshall Islands,437.20,714043796,2/12/2020


In [14]:
data_df_ge = ge.dataset.PandasDataset(data_df)

In [ ]:
data_df_ge['Order Date'] = pd.to_datetime(data_df_ge['Order Date']).dt.strftime("%Y/%m/%d")

In [ ]:
data_df_ge['Unit Price'].map('{:.4f}'.format)

In [ ]:
data_df_ge2 = data_df_ge[:3]

In [ ]:
type(data_df_ge2)

In [ ]:
data_df_ge2

In [ ]:
data_df_ge2.expect_column_values_to_be_in_set('Country',list(data_df_ge.Country),result_format={'result_format':'COMPLETE'})